In [1]:
import pandas as pd
import numpy as np
import os

from pathlib import Path

raw_data_path = Path('../data/raw')
output_folder_path = Path('../data/processed')

In [2]:
os.listdir('../data/raw')

['county_asthma_prevalence.csv',
 'oz_county_avg_2016-2020.csv',
 'us_fips_codes.csv',
 'pm25_county_avg_2016-2020.csv',
 'asthma_ed_visit.csv',
 'county_populations_2010-2019.xlsx',
 '.ipynb_checkpoints',
 'pm25_county_avg.csv',
 'oz_county_avg.csv']

---

### FIPS codes

- source: https://github.com/kjhealy/fips-codes/blob/master/state_and_county_fips_master.csv

In [56]:
us_fips = pd.read_csv(os.path.join(raw_data_path, 'us_fips_codes.csv'))
ca_fips = us_fips[us_fips['state'] == 'CA'].copy()
ca_fips['name'] = ca_fips['name'].str.extract(r'(.+) County')
ca_fips = ca_fips.rename(columns={'name':'county'})

In [57]:
ca_fips.head()

,fips,county,state
192,6001,Alameda,CA
193,6003,Alpine,CA
194,6005,Amador,CA
195,6007,Butte,CA
196,6009,Calaveras,CA


In [58]:
#ca_fips.to_csv(path_or_buf= os.path.join(output_folder_path, 'ca_fips.csv'), index=False)

---

### PM2.5 Data

In [8]:
pm_data = pd.read_csv(os.path.join(raw_data_path, 'pm25_county_avg.csv'))
pm_data['countyfips'] = pm_data['countyfips'] + 6000

In [9]:
pm_data.head()

,countyfips,DS_PM_pred,DS_PM_stdd
0,6001,10.095240,3.627715
1,6101,10.718292,4.210715
2,6103,9.015758,3.753104
3,6105,6.894584,3.502783
4,6107,15.517773,6.683213


In [16]:
# pm_data.to_csv(path_or_buf= os.path.join(output_folder_path, 'pm_data.csv'), index=False)

---

### Ozone Data

In [11]:
oz_data = pd.read_csv(os.path.join(raw_data_path, 'oz_county_avg.csv'))
oz_data['countyfips'] = oz_data['countyfips'] + 6000

In [12]:
oz_data.head()

,countyfips,DS_O3_pred,DS_O3_stdd
0,6001,31.868148,3.520695
1,6003,46.396664,5.339098
2,6005,44.023391,4.515013
3,6007,42.151413,4.400108
4,6009,44.858142,4.677322


In [17]:
# oz_data.to_csv(path_or_buf= os.path.join(output_folder_path, 'oz_data.csv'), index=False)

---

### County Populations (2016 - 2019)

In [14]:
county_pop = pd.read_excel(os.path.join(raw_data_path, 'county_populations_2010-2019.xlsx'), skiprows=3, nrows=59)
county_pop = county_pop.rename(columns={'Unnamed: 0': 'county'}).drop(0)
county_pop['county'] = county_pop['county'].str.extract(r'.(.+) County,')
county_pop = county_pop.replace(to_replace=r'[,.]', value='', regex=True)

years = [2016, 2017, 2018, 2019]
for year in years:
    county_pop[year] = pd.to_numeric(county_pop[year], errors='coerce')

county_pop['avg_population'] = county_pop[years].mean(axis=1)
county_pop = county_pop[['county', 'avg_population']]

county_pop.head()

,county,avg_population
1,Alameda,1662307.75
2,Alpine,1094.00
3,Amador,38778.75
4,Butte,226113.00
5,Calaveras,45651.50


In [18]:
# county_pop.to_csv(path_or_buf= os.path.join(output_folder_path, 'ca_county_populations_2016-2019.csv'), index=False)

---

### Asthma Prevalences

In [20]:
prevalences = pd.read_csv(os.path.join(raw_data_path, 'county_asthma_prevalence.csv'))

prevalences = prevalences[(prevalences['STRATA'] == 'Total population') & \
            (prevalences['AGE GROUP'] == 'All ages') & \
            (prevalences['YEARS'].isin(['2015–2016', '2017–2018', '2019–2020'])) & \
            (prevalences['COUNTY'] != 'California')]

prevalences = prevalences.rename(columns={'COUNTY': 'county',
                                          'YEARS': 'years',
                                          'AGE GROUP': 'age_group',
                                          'CURRENT PREVALENCE': 'asthma_prevalence',
                                          'COUNTIES GROUPED': 'counties_grouped'})

prevalences = prevalences[['county', 'years', 'age_group', 'asthma_prevalence', 'counties_grouped']]
prevalences = pd.DataFrame(prevalences.groupby('county')['asthma_prevalence'].mean().rename('avg_asthma_prevalence')).reset_index()

prevalences.head()

,county,avg_asthma_prevalence
0,Alameda,8.466667
1,Alpine,9.933333
2,Amador,9.933333
3,Butte,11.233333
4,Calaveras,9.933333


In [22]:
# prevalences.to_csv(path_or_buf= os.path.join(output_folder_path, 'ca_asthma_prevalences_2015-2020.csv'), index=False)

---

### Asthma ED Visit Counts

- Values are noted as a rate per 10,000 residents

In [43]:
ed_visits = pd.read_csv(os.path.join(raw_data_path, 'asthma_ed_visit.csv'))

ed_visits = ed_visits[(ed_visits['STRATA'] == 'Total population') & \
            (ed_visits['AGE GROUP'] == 'All ages') & \
            (ed_visits['YEAR'].isin([2015, 2016, 2017, 2018, 2019])) & \
            (ed_visits['COUNTY'] != 'California')]

ed_visits = ed_visits.rename(columns={'COUNTY':'county',
                                      'NUMBER OF ED VISITS': 'num_ed_visits'})

ed_visits = ed_visits[['county', 'num_ed_visits']]
ed_visits['num_ed_visits'] = ed_visits['num_ed_visits'].astype(str).replace(to_replace='[,]', value='', regex=True).astype(float)
ed_visits = pd.DataFrame(ed_visits.groupby('county')['num_ed_visits'].mean().rename('avg_number_ed_visits')).reset_index()

ed_visits.head()

,county,avg_number_ed_visits
0,Alameda,8642.0
1,Alpine,0.0
2,Amador,173.0
3,Butte,882.0
4,Calaveras,187.8


In [26]:
# ed_visits.to_csv(path_or_buf= os.path.join(output_folder_path, 'ca_ed_visit_counts_2015-2019.csv'), index=False)

---

---

### Combining Datasets

In [105]:
county_populations = pd.merge(left=county_pop, right=ca_fips, on='county')
county_prevalences = pd.merge(left=county_populations, right=prevalences, on='county')
county_prevalences['num_asthma_cases'] = (county_prevalences['avg_asthma_prevalence'] / 100) * county_prevalences['avg_population']

full_dataset = pd.merge(left=county_prevalences, right=ed_visits, on='county').drop(columns='state')
full_dataset = pd.merge(left=full_dataset, right=pm_data, left_on='fips', right_on='countyfips').drop(columns='countyfips')
full_dataset = pd.merge(left=full_dataset, right=oz_data, left_on='fips', right_on='countyfips').drop(columns='countyfips')


In [107]:
full_dataset.head()

,county,avg_population,fips,avg_asthma_prevalence,num_asthma_cases,avg_number_ed_visits,DS_PM_pred,DS_PM_stdd,DS_O3_pred,DS_O3_stdd
0,Alameda,1662307.75,6001,8.466667,140742.056167,8642.0,10.095240,3.627715,31.868148,3.520695
1,Alpine,1094.00,6003,9.933333,108.670667,0.0,6.623339,2.938605,46.396664,5.339098
2,Amador,38778.75,6005,9.933333,3852.022500,173.0,8.862555,3.681368,44.023391,4.515013
3,Butte,226113.00,6007,11.233333,25400.027000,882.0,10.568623,4.342973,42.151413,4.400108
4,Calaveras,45651.50,6009,9.933333,4534.715667,187.8,9.474165,3.958912,44.858142,4.677322


In [108]:
full_dataset.to_csv(path_or_buf= os.path.join(output_folder_path, 'full_dataset.csv'), index=False)